In [1]:
# import all packages to use in this project

import os
import pandas as pd
import numpy as np
import geopandas as gpd
import json
import glob
import natsort
import shutil
from glom import glom

/home/yuri/.virtualenvs/firelocenv/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
from glass.sql.db import create_pgdb
from glass.sql.tbl import create_tbl
from glass.dtt.mge.pd import merge_df
from glass.rd.shp import shp_to_obj
from glass.prop.sql import lst_tbl
from glass.wt.sql import df_to_db

In [3]:
###### Variáveis a serem alteradas ######

stations = '/home/yuri/fireloc-system/firerisk/data/stations.geojson'
observations = '/home/yuri/fireloc-system/firerisk/data/'
portugal = '/home/yuri/fireloc-system/firerisk/data/caop_pt.shp'
db = 'ipma2'

#########################################

In [4]:
datab = create_pgdb(db)

In [5]:
tbl_varia = {
    'variavel' : {'id' : 'int', 'nome' : 'varchar(30)'}
}

In [6]:
tbl_sta = {
    'estacao': {'id' : 'int', 'idEstacao' : 'int', 'localEstacao' : 'varchar(80)', 'geom' : 'geometry'}
}

In [7]:
tbl_values = {
    'valor' : {'id' : 'int', 'id_est' : 'int', 'id_var' : 'int', 'daytime' : 'varchar(20)', 'valor' : 'float'}
}

In [8]:
tbl_v = create_tbl(db, tbl_varia)

In [9]:
tbl_e = create_tbl(db, tbl_sta)

In [10]:
tbl_vl = create_tbl(db, tbl_values)

In [11]:
pt = shp_to_obj(portugal)

In [12]:
pt_dis = pt.unary_union

In [13]:
pt_diss = gpd.GeoDataFrame(geometry=[pt_dis], crs = 'EPSG:3763')

In [14]:
sdata = json.load(open(stations, 'r'))

In [15]:
gjson = {"type": "FeatureCollection", "features" : sdata}

In [16]:
sdf= gpd.GeoDataFrame.from_features(gjson, crs="EPSG:4326").to_crs(3763)
#sdf_crs = sdf.to_crs(3763)

In [17]:
sdf_pt = sdf.clip(pt_diss)

In [18]:
sdf_pt.index = np.arange(1, len(sdf_pt) + 1)

In [19]:
sdf_pt['id'] = sdf_pt.index

/home/yuri/.virtualenvs/firelocenv/lib/python3.10/site-packages/geopandas/geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [20]:
variaveis = [
    'intensidadeVentoKM', 'temperatura', 'radiacao', 'idDireccVento',
    'precAcumulada', 'intensidadeVento', 'humidade', 'pressao'
]

In [21]:
varlst = [[i + 1, variaveis[i]] for i in range(len(variaveis))]

variaveis = pd.DataFrame(varlst, columns=['id', 'nome'])

In [22]:
obser = glob.glob(observations + '**.json')

In [26]:
for o in obser:
    odata = json.load(open(o, 'r'))
    
    obs = []
    
    for d in odata:
        for station in odata[d]:
            if not odata[d][station]: continue

            odata[d][station]['daytime']= d
            odata[d][station]['idEstacao']= station

            obs.append(odata[d][station])

    df=pd.DataFrame(obs)
    
    cols = variaveis.nome.tolist()

    dfs = []

    df_cols = df.columns.values
    icols = ['idEstacao', 'daytime']
    for col in cols:
        _df = df.copy(deep=True)

        _df.drop([c for c in df_cols if c != col and c not in icols], axis=1, inplace=True)

        _df.rename(columns={col: 'valor'}, inplace=True)

        _df['varid'] =col

        dfs.append(_df)
        
    _all = merge_df(dfs)
        
    _all['id_var'] = 0
    
    for i, r in variaveis.iterrows():
        _all['id_var'] = np.where(
            _all.varid == r['nome'],
            r['id'], _all['id_var']
        )
    
    _all['idEstacao'] = pd.to_numeric(_all['idEstacao'])
    
    sdf_pt['idEstacao'] = pd.to_numeric(sdf_pt['idEstacao'])
    
    _all['id_est'] = 0

    for i, r in sdf_pt.iterrows():

        _all['id_est'] = np.where(
            _all.idEstacao == r['idEstacao'],
            r['id'], _all['id_est']
        )
    
    fdf = _all.loc[(_all["id_est"] > 0)]
    
    fdf.drop('varid', axis=1, inplace=True)
    fdf.drop('idEstacao', axis=1, inplace=True)
    
    fdf.index = np.arange(1, len(fdf) + 1)
    fdf['id'] = fdf.index
    
    variaveis = variaveis[['id', 'nome']]
    sdf_pt = sdf_pt[['id', 'idEstacao', 'localEstacao', 'geometry']]
    fdf = fdf[['id', 'id_var', 'id_est', 'daytime', 'valor']]
    
    df_db_v = df_to_db(db, variaveis, "variavel")
    df_db_st = df_to_db(db, sdf_pt, "estacao", epsg=3763, geomType = "Point")
    df_db_vl = df_to_db(db, fdf, "valor")
    print(fdf)

/tmp/ipykernel_3179332/1127790736.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fdf.drop('varid', axis=1, inplace=True)
/tmp/ipykernel_3179332/1127790736.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fdf.drop('idEstacao', axis=1, inplace=True)
/tmp/ipykernel_3179332/1127790736.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fdf['id'] = fdf.index


          id  id_var  id_est           daytime   valor
1          1       1      54  2023-03-06T00:00     2.2
2          2       1      63  2023-03-06T00:00     6.5
3          3       1       1  2023-03-06T00:00     2.2
4          4       1      49  2023-03-06T00:00     6.1
5          5       1      48  2023-03-06T00:00     7.6
...      ...     ...     ...               ...     ...
25756  25756       8     105  2023-03-06T02:00   -99.0
25757  25757       8      57  2023-03-06T02:00   -99.0
25758  25758       8     138  2023-03-06T02:00   -99.0
25759  25759       8      14  2023-03-06T02:00   -99.0
25760  25760       8      16  2023-03-06T02:00  1015.9

[25760 rows x 5 columns]


/tmp/ipykernel_3179332/1127790736.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fdf.drop('varid', axis=1, inplace=True)
/tmp/ipykernel_3179332/1127790736.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fdf.drop('idEstacao', axis=1, inplace=True)
/tmp/ipykernel_3179332/1127790736.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fdf['id'] = fdf.index


          id  id_var  id_est           daytime   valor
1          1       1      54  2023-03-06T00:00     2.2
2          2       1      63  2023-03-06T00:00     6.5
3          3       1       1  2023-03-06T00:00     2.2
4          4       1      49  2023-03-06T00:00     6.1
5          5       1      48  2023-03-06T00:00     7.6
...      ...     ...     ...               ...     ...
25756  25756       8     105  2023-03-06T02:00   -99.0
25757  25757       8      57  2023-03-06T02:00   -99.0
25758  25758       8     138  2023-03-06T02:00   -99.0
25759  25759       8      14  2023-03-06T02:00   -99.0
25760  25760       8      16  2023-03-06T02:00  1015.9

[25760 rows x 5 columns]


In [24]:
fdf

,id,id_var,id_est,daytime,valor
1,1,1,54,2023-03-06T00:00,2.2
2,2,1,63,2023-03-06T00:00,6.5
3,3,1,1,2023-03-06T00:00,2.2
4,4,1,49,2023-03-06T00:00,6.1
5,5,1,48,2023-03-06T00:00,7.6
...,...,...,...,...,...
25756,25756,8,105,2023-03-06T02:00,-99.0
25757,25757,8,57,2023-03-06T02:00,-99.0
25758,25758,8,138,2023-03-06T02:00,-99.0
25759,25759,8,14,2023-03-06T02:00,-99.0
